In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from copy import deepcopy
from label_manipulation import median_filter_labels
import tensorflow as tf


In [2]:
from SignalRecognition.cwt_learner.wavelet_feature_engineering import CWT_learner
from signal_database.labelSelectedData import SignalDB,SignalBundle

In [4]:
import matplotlib.pyplot as plt
from plot_generator import plotResult_colorbars

In [6]:
sdb = SignalDB('grab_objects', path='./')
training_data_ = sdb.get_labeleddata()

In [7]:
%matplotlib

Using matplotlib backend: TkAgg


In [8]:
plt.figure()
for ii,example in enumerate(training_data_):
    plt.subplot(len(training_data_)*2,1,ii*2 + 1)
    for channel in example.signal_bundle.signals:
        plt.plot(channel)
    plt.subplot(len(training_data_)*2,1,ii*2 + 2)
    labels = example.labels
    plotResult_colorbars(labels, range(len(labels)))
plt.show()

In [19]:
all_possible_labels = []
for example in training_data_:
        all_possible_labels = all_possible_labels + example.labels
print set(all_possible_labels)



set(['', 'freespace', 'hold_foamblock', 'grab_hex', 'hand_on', 'grab_cup', 'release_foamblock', 'release_cup', 'hold_cup', 'grab_foamblock', 'still', 'hold_hex', 'release_hex', 'hand_off'])


In [9]:
training_data_relabeled = deepcopy(training_data_)


In [10]:
# label map 
label_map = {
    "freespace" : "freespace",
    "grab_hex" : "grab",
    "grab_cup" : "grab",
    "grab_foamblock" : "grab",
    "release_foamblock" : "release",
    "release_cup" : "release",
    "release_hex" : "release",
    "hold_cup" : "hold",
    "hold_hex" : "hold",
    "hold_foamblock" : "hold",
    "hand_on" : "hand_on",
    "hand_off" : "hand_off",
    "still" : "still",
    "" : ""
}



for example in training_data_relabeled:
        example.labels = [label_map[label] for label in example.labels]


In [12]:
cwt_learn = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in training_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn.add_training_data(signals,labels)

In [13]:
all_labels = []
for example in training_data_relabeled:
        all_labels = example.labels + all_labels
label_types = list(set(all_labels))
print label_types

['', 'freespace', 'hold', 'hand_off', 'hand_on', 'release', 'grab', 'still']


In [14]:
(trainX,trainY) = cwt_learn.get_examples_with_weights(cwt_learn.training_data_sets,with_window = "Off",
                                                         signal_indices = cwt_learn.signal_indices
                                                         ,wavelet = cwt_learn.wavelet)




In [29]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from scipy.interpolate import interp1d

In [53]:
unknown_samples = [ii for ii,label in enumerate(trainY) if label == '']
known_samples = [ii for ii,label in enumerate(trainY) if label != '']
known_labels = [label for ii,label in enumerate(trainY) if label != '']


label_names_uniques,known_labels_ids = np.unique(known_labels,return_inverse=True)


In [61]:
f1 = interp1d(known_samples, known_labels_ids, kind='nearest',fill_value='extrapolate')
unknown_labels_close_enough_ids = f1(unknown_samples)
unknown_labels_close_enough = [label_names_uniques[int(label_id)] for label_id in unknown_labels_close_enough_ids]

In [75]:
current_id = 0
pl1 = 0
pl2 = 0
out_list = []
for ii in range(len(trainY)):
    if unknown_samples[pl1] == current_id:
        out_list.append(unknown_labels_close_enough[pl1])
        pl1 += 1
    else:
        out_list.append(known_labels[pl2])
        pl2 +=1
    current_id += 1
print out_list 

In [92]:
def fill_closest(labels,replace = ""):
    unknown_samples = [ii for ii,label in enumerate(labels) if label == replace]
    known_samples = [ii for ii,label in enumerate(labels) if label != replace]
    known_labels = [label for ii,label in enumerate(labels) if label != replace]
    label_names_uniques,known_labels_ids = np.unique(known_labels,return_inverse=True)
    f1 = interp1d(known_samples, known_labels_ids, kind='nearest',fill_value='extrapolate')
    unknown_labels_close_enough_ids = f1(unknown_samples)
    unknown_labels_close_enough = [label_names_uniques[int(label_id)] for label_id in unknown_labels_close_enough_ids]
    current_id = 0
    pl1 = 0
    pl2 = 0
    out_list = []
    for ii in range(len(labels)):
        if unknown_samples[pl1] == current_id:
            out_list.append(unknown_labels_close_enough[pl1])
            pl1 += 1
        else:
            out_list.append(known_labels[pl2])
            pl2 +=1
        current_id += 1
    return out_list

trainY_filled = fill_closest(trainY)

In [213]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [143]:
Y_one_hot = pd.get_dummies(trainY_filled)
print np.shape(trainX)[1], np.shape(Y_one_hot)

80 (367130, 7)


In [150]:
label_names_df = list(Y_one_hot.columns.values)

In [149]:
# np.array(Y_one_hot.idxmax(axis=1))

array(['still', 'still', 'still', ..., 'still', 'still', 'still'],
      dtype=object)

In [129]:
def baseline_model():
    model = Sequential()
    model.add(Dense(10,input_dim=np.shape(trainX)[1],activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(np.shape(Y_one_hot)[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# estimator = KerasClassifier(build_fn=baseline_model, epochs=1, batch_size=1000, verbose=1)

In [241]:
def lstm_model():
    model = Sequential()
    model.add(Dense(10,input_dim=np.shape(trainX)[1],activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(LSTM(10,dropout_U = 0.2, dropout_W = 0.2))
    model.add(Dense(np.shape(Y_one_hot)[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [244]:
Y_one_hot = np.array(Y_one_hot)
trainX = np.array(trainX)
trainX_LSTM = np.reshape(trainX,(np.shape(trainX)[0],np.shape(trainX)[1],1))


In [247]:
model = lstm_model()
print model.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, dropout=0.2, recurrent_dropout=0.2)`
  """


ValueError: Input 0 is incompatible with layer lstm_12: expected ndim=3, found ndim=2

In [246]:

model.fit(trainX_LSTM,Y_one_hot,epochs = 20, batch_size=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, dropout=0.2, recurrent_dropout=0.2)`
  """


ValueError: Input 0 is incompatible with layer lstm_11: expected ndim=3, found ndim=2

In [171]:
cwt_learn_test = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in testing_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn_test.add_training_data(signals,labels)
    
(testX,testY) = cwt_learn_test.get_examples_with_weights(cwt_learn_test.training_data_sets,with_window = "Off",
                                                         signal_indices = cwt_learn_test.signal_indices
                                                         ,wavelet = cwt_learn_test.wavelet)

In [194]:
testX = np.array(testX)

In [195]:
# plt.subplot(2,1,1)
# plotResult_colorbars(out_list, range(len(out_list)),labelNames=label_types)
# plt.subplot(2,1,2)
# plotResult_colorbars(trainY, range(len(trainY)),labelNames=label_types)

In [196]:
test_one_hot_predict = model.predict(testX)


In [197]:
test_one_hot_predict = (test_one_hot_predict > 0.9).astype(int)

In [198]:
test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)

In [206]:
test_one_hot_predict_series_ = list(test_one_hot_predict_df.idxmax(axis = 1))

In [207]:
test_one_hot_predict_series = median_filter_labels(test_one_hot_predict_series_,window=1001)

In [208]:
ax = plt.subplot(2,1,1)

plotResult_colorbars(test_one_hot_predict_series, range(len(test_one_hot_predict_series)),
                     labelNames=label_types,ax=ax)
ax.set_xlim([0,len(test_one_hot_predict_series)])
ax = plt.subplot(2,1,2)
plotResult_colorbars(testY, range(len(testY)),labelNames=label_types,ax=ax)
ax.set_xlim([0,len(test_one_hot_predict_series)])


(0, 198932)

In [ ]:
## stuff after this is backup

In [48]:
fit_labels = []

for ii,ld in enumerate(testing_data):
    
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    labels = cwt_learn.fit(signals)
    fit_labels.append(labels)
    

In [54]:
plt.figure()
for ii,(ld,fit_label) in enumerate(zip(testing_data,fit_labels)):
    
    plt.subplot(len(testing_data)*2,1,ii*2 + 1)
    plotResult_colorbars(fit_label, range(len(fit_label)),labelNames=label_types)
    plt.subplot(len(testing_data)*2,1,ii*2 + 2)
    plotResult_colorbars(ld.labels, range(len(labels)),labelNames=label_types)
plt.show()

In [52]:
for ii,(ld,fit_label) in enumerate(zip(testing_data,fit_labels)):
    print len(ld.labels),len(fit_label)    



37855 37855
38059 38059
39893 39893
34573 34573
48552 48552


In [92]:
filtered_labels = median_filter_labels(fit_label,1001)
plt.subplot(2,1,1)
plotResult_colorbars(filtered_labels, range(len(labels)),labelNames=label_types)
plt.subplot(2,1,2)
plotResult_colorbars(ld.labels, range(len(labels)),labelNames=label_types)